<a href="https://colab.research.google.com/github/DustinHaschke/Hangman/blob/main/Hangman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from os import truncate
from numpy.lib.function_base import append
from numpy.core.numeric import False_
import numpy as np
from IPython.display import clear_output
import time


# Importieren des Wörterbuchs
r = requests.get("https://gist.githubusercontent.com/MarvinJWendt/2f4f4154b8ae218600eb091a5706b5f4/raw/36b70dd6be330aa61cd4d4cdfda6234dcb0b8784/wordlist-german.txt")
data = r.text.split("\n")

# upper-Funktion inklusive des 'ß'
def all_upper(zeichen):
    if zeichen == "ẞ" or zeichen == "ß":
        return "ẞ"
    else:
        return zeichen.upper()

# Rückgabe aller Substantive der übergebenen Länge aus Wörterbuch
def woerter_liste(laenge):
  liste = []
  if laenge > 45:
    laenge = 45
  while liste == []:
    for i in range(len(data)):
        if len(data[i]) == laenge and data[i][0].isupper():
            liste.append(data[i])
    laenge -= 1
  return liste

# "zufällige" Generierung eines zu erratenden Wortes
def wort_generator(liste):
    return liste[np.random.randint(0, len(liste)-1)]

# Lebensanzeige: a_0 - Urprungsanzahl, a_neu - noch verbleibende Leben
def leben_anzeige(a_0,a_neu):
    print('\033[91m' + a_neu * '\u2665 ' + (a_0 - a_neu) * '\u2661 ' + '\033[0m')

# Ratetafel des Wortes
def ratetafel(wort,gewaehlte):
    ausgabe = ""
    for zeichen in wort:
        if all_upper(zeichen) in gewaehlte:
            ausgabe += zeichen + " "
        else:
            ausgabe += "_ "
    return ausgabe

# Noch ungewählte Buchstaben
def abc_uebersicht(alphabet,gewaehlte):
    ausgabe =""
    for buchstabe in alphabet:
        if buchstabe in gewaehlte:
            ausgabe += "_ "
        else:
            ausgabe += buchstabe + " "
    return ausgabe

# Anlegen der Parameter
gewaehlte_buchstaben= []
alphabet = ["A","B","C","D","E","F","G","H","I","J","K","L","M","N","O","P","Q","R","S","T","U","V","W","X","Y","Z","Ä","Ö","Ü","ẞ"]
geloest = False
error_buchstabe = False
resultat_letztes_raten = "Viel Spaß bei Hangman :-)"
runden = 1


# Interaktion Wortmodus
print("Willst du... \n1) ein Wort vorgeben oder soll \n2) ein Wort generiert werden? \n Gib 1 oder 2 ein:")
wort_modus = input()
clear_output()

# Schleife, falls kein Wortmodus gewählt
while (wort_modus != "1" and wort_modus != "2"):
    print("Ich habe deine Eingabe leider nicht verstanden. \n\nWillst du... \n1 | ein Wort vorgeben oder soll \n2 | ein Wort generiert werden? \n Gib 1 oder 2 ein:")
    wort_modus = input()
    clear_output()

# Wort manuell wählen
if wort_modus == "1":
    print("Gib dein Wort ein (keine Zahlen & Sonderzeichen!):")
    wort = input()
    laenge = int(len(wort))
    clear_output()

# Wort aus Wörterbuch
else:
    print("Wie viele Buchstaben soll dein Wort enthalten?")
    laenge = int(input())
    while laenge < 1:
        print("Ein Wort dieser Länge existiert leider nicht. Wähle bitte noch einmal:")
        laenge = input()
    wort = wort_generator(woerter_liste(int(laenge)))
    clear_output()
    laenge = len(wort)

# Lebenanzahl wird durch Wörterlänge definiert und Zeit startet
leben_max = max(5,8-laenge//10)
leben = leben_max
startzeitpunkt = time.time()

# Spielbeginn
while (geloest == False and leben != 0):

  # print(wort)                        # Cheat-Ausgabe zu Testzwecken
  print ("Runde", runden,"  |   Wortlänge",laenge ,"\n")
  print(resultat_letztes_raten, "\n")
  tafel = str(ratetafel(wort,gewaehlte_buchstaben))
  print('\033[92m' + tafel + '\033[0m')
  leben_anzeige(leben_max,leben)
  print("")
  print(abc_uebersicht(alphabet,gewaehlte_buchstaben))

  # Eingabe des neuen Buchstaben + Text
  neuer_buchstabe = input("Gib einen neuen Buchstaben ein:")
  neuer_buchstabe = all_upper(str(neuer_buchstabe))

  # Prüfung, ob neuer Buchstabe in Alphabet ist
  error_buchstabe = True
  resultat_letztes_raten = "Den Buchstaben   " + neuer_buchstabe + "   kenne ich leider nicht."
  for eintrag in alphabet:
    if eintrag == neuer_buchstabe:
      error_buchstabe = False

  # Prüfung, ob neuer Buchstabe bereits gewählt wurde
  for eintrag in gewaehlte_buchstaben:
    if eintrag == neuer_buchstabe:
      error_buchstabe = True
      resultat_letztes_raten = "Den Buchstaben   " + neuer_buchstabe + "   hast du bereits gewählt!"

  # Buchstabe ordnungsgemäß gewählt
  if not(error_buchstabe):
    gewaehlte_buchstaben.append(neuer_buchstabe)
    neue_tafel = ratetafel(wort,gewaehlte_buchstaben)

      # falls Buchstabe nicht im Wort
    if tafel == neue_tafel:
      leben -= 1
      resultat_letztes_raten = "Der Buchstabe   " + neuer_buchstabe + "   war leider falsch."

    # falls Buchstabe im Wort
    else:
      resultat_letztes_raten = "Der Buchstabe   " + neuer_buchstabe + "   war richtig. Toll gemacht!"
      if neue_tafel.count('_') == 0:
        geloest = True

  # Rundenzähler, Print-Aktualisierer und Anti-Bug
  time.sleep(0.1)
  runden += 1
  clear_output()
  time.sleep(0.1)

# Sieges- oder Niederlagen-Ausgabe
if geloest:
  benoetigte_zeit = round(time.time() - startzeitpunkt)
  print ("Runde", runden,"  |   Wortlänge",laenge ,"\n")
  print(str('\033[94m' + ratetafel(wort,gewaehlte_buchstaben) + '\033[0m'))
  leben_anzeige(leben_max,leben)
  print(abc_uebersicht(alphabet,gewaehlte_buchstaben))
  print ('\033[92m' + "\nGlückwunsch, du hast in nur", runden, "Runden und",benoetigte_zeit, "Sekunden gewonnen!!! :-D" + '\033[0m')
else:
  print('\033[95m' + "Oh nein, leider hast du alle Leben verloren.\n\nDas gesuchte Wort wäre   " + '\033[0m', '\033[94m' + wort + '\033[0m' , '\033[95m' + "   gewesen.\nVersuche es doch noch einmal. :-)" + '\033[0m')

Runde 16   |   Wortlänge 45 

M a s c h i n e n b e t r i e b s u n t e r b r e c h u n g s v e r s i c h e r u n g e n 
♥ ♥ ♥ ♥ ♡ 
_ _ _ D _ _ _ _ _ J K L _ _ O P Q _ _ _ _ _ W X Y Z Ä Ö Ü ẞ 

Glückwunsch, du hast in nur 16 Runden und 68 Sekunden gewonnen!!! :-D


In [ ]:
roboter = ["C","R"][6%5] + str(["3","P","2"][2]) + ["A",str("D"),"O"][bool(2%2==0)] + str(2*int(True))
print(roboter)

R2D2
